# Configurar Entorno

## Importar librerias

In [29]:
# Filtramos los warnings
import warnings
warnings.filterwarnings("ignore")

# Forzar uso CPU en caso de dispones GPU en el PC
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# instalar librerias
#!pip3 install gym gym-retro
#Instalar con pip3, con pip dio problemas al reconocer las roms
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip3 install optuna
#!pip3 install tensorflow==2.3.0
#!pip3 install keras keras-rl2

# Import retro para crear el entorno Street Fighter a partir de la ROM
import retro

# Import Time para relentizar el juego
import time

# Import Clase base del entorno para hacer wrapper
from gym import Env

# Import los shapes espaciales para el entorno
from gym.spaces import MultiBinary, Box

# Import numpy para calcular el frame delta
import numpy as np

# Import opencv para aplicar la escala de grises
import cv2

# Import matplotlib para visualizar la imagen
from matplotlib import pyplot as plt

# Import generación modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

# Construcción del agente
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

## Configuración del Entorno

In [30]:
# Creamos una clase para definir el entorno de SF2
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        # Especificar el espacio de acciones y el espacio de observaciones
        self.observation_space = Box(low=0, 
                                     high=255, 
                                     shape=(84, 84, 1), 
                                     dtype=np.uint8)
        
        self.action_space = MultiBinary(12)
        
        # Instanciar el entorno
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis',
                               use_restricted_actions=retro.Actions.FILTERED)
    
    def reset(self):
        
        # Devolvemos el primer frame
        obs = self.game.reset()
         
        # Preprocess
        obs = self.preprocess(obs)
        self.previous_frame = obs
        
        # Inicializar atributo para la diferencia de la puntuación
        self.score = 0
        
        return obs
    
    def preprocess(self, observation):
        # Aplicamos el redimensionado del frame y el escalado de grises
        # Escalado de grises
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Redimensionado del frame
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Añadir el valor de los canales
        channel = np.reshape(resize, (84,84,1))
        
        return channel
    
    def step(self, action):
        # Realizar una acción
        obs, reward, done, info = self.game.step(action)
        
        # Procesamos la observación
        obs = self.preprocess(obs)
        
        # Calcular frame delta (Variación en frame anterior y actual)
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        
        # Adaptamos la función de recompensa
        reward = info['score'] - self.score
        self.score = info['score']
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

# Creación modelo

La arquitectura de la DQN se compondrá de dos partes:
 * Visión por computación: reconocimiento de imágenes.
 * Regresión: determinar los valores Q de cada acción.

In [45]:
# Definición de la arquitectura de la DQN

def build_DQN(input_shape, output_shape):
    
    model = Sequential()
    
    # Bloque Capas convolucionales
    model.add(Conv2D(input_shape=input_shape, filters=32, kernel_size=8, strides=4, activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=4, strides=2, activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, activation="relu"))
    
    model.add(Flatten())
    
    # Bloque Capas lineales para determinar el Q-valor 
    model.add(Dense(units=512, activation='linear'))
    model.add(Dense(units=output_shape, activation='linear'))
    
    return model

In [46]:
env.close()
env = StreetFighter()

In [47]:
model = build_DQN(env.observation_space.shape, env.action_space.shape[0])

In [48]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 20, 20, 32)        2080      
                                                                 
 conv2d_6 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 512)               1606144   
                                                                 
 dense_1 (Dense)             (None, 12)                6156      
                                                                 
Total params: 1,684,140
Trainable params: 1,684,140
No